In [1]:
import numpy as np

ModuleNotFoundError: No module named 'numpy'

<p>G= (T, N, R,S)</p>
<p>Conjunto de terminales: T= {time, flies, arrow, an, like}</p>
<p>Conjunto de no terminales: N= {S, NP, VP, PP, Det, Nominal, Verb, Preposition, Noun}</p>
<p>
Conjunto de reglas:
R={
<p>S → NP VP				0.800
<p>NP → time | flies | arrow		0.002
<p>NP → Det Nominal			0.300
<p>NP → Nominal Nominal		0.200
<p>Nominal → time | flies | arrow	0.002
<p>Nominal → Nominal Noun		0.100
<p>Nominal → Nominal PP		0.200
<p>VP → time				0.004
<p>VP → flies| like			0.008
<p>VP → Verb NP				0.300
<p>VP → Verb PP				0.200
<p>PP → Preposition NP			0.100
<p>Verb → time				0.010
<p>Verb → flies| like			0.020
<p>Noun → time | flies | arrow		0.010
<p>Det → an				0.050
<p>Preposition → like			0.050
}
</p>


In [1]:
# Establecemos las reglas con diccionarios:
Reglas = {  
    
  'time': {'NP': 0.002, 'Nominal': 0.002, 'VP': 0.004, 'Verb': 0.010, 'Noun': 0.010},
  
  'flies':{'NP': 0.002, 'Nominal': 0.002, 'VP': 0.008, 'Verb': 0.020, 'Noun': 0.010},
  
  'like':{'VP': 0.008, 'Verb': 0.020, 'Preposition': 0.050},

  'an':{'Det': 0.050},

  'arrow': {'NP': 0.002, 'Nominal': 0.002, 'Noun': 0.010},

  'Det': {
      'Nominal':{'NP':0.300}
  },

  'NP': {
      'VP':{'S':0.800}
  },

  'Nominal': {
      'Nominal':{'NP':0.200},
      'Noun': {'Nominal': 0.100},
      'PP': {'Nominal': 0.200}
    },
     
  'Verb':{
      'NP':{'VP':0.300},
      'PP':{'VP':0.200}
  },

  'Preposition':{
      'NP':{'PP':0.100}
  }     

}



In [3]:
frase = "time flies like an arrow"

Palabras = frase.split(' ')

cantidad_palabras = len(Palabras)

for palabra in Palabras:
  print(palabra, Reglas[palabra])


time {'NP': 0.002, 'Nominal': 0.002, 'VP': 0.004, 'Verb': 0.01, 'Noun': 0.01}
flies {'NP': 0.002, 'Nominal': 0.002, 'VP': 0.008, 'Verb': 0.02, 'Noun': 0.01}
like {'VP': 0.008, 'Verb': 0.02, 'Preposition': 0.05}
an {'Det': 0.05}
arrow {'NP': 0.002, 'Nominal': 0.002, 'Noun': 0.01}


In [4]:
# Declaramos un arreglo de 5 x 5 (palabras en la frase), de tipo dict de tal forma que pueda guardar entradas con las reglas
Matriz = np.zeros(shape=(cantidad_palabras, cantidad_palabras), dtype=dict)


In [5]:
# Recorremos la frase de izquiera a derecha para ir calculando los valores de la diagonal principal
for col in range(cantidad_palabras):
  palabra = Palabras[col]
  for fila in range(cantidad_palabras):
      if (fila == col): Matriz[fila, col] = Reglas[palabra]

In [6]:
Matriz

array([[{'NP': 0.002, 'Nominal': 0.002, 'VP': 0.004, 'Verb': 0.01, 'Noun': 0.01},
        0, 0, 0, 0],
       [0,
        {'NP': 0.002, 'Nominal': 0.002, 'VP': 0.008, 'Verb': 0.02, 'Noun': 0.01},
        0, 0, 0],
       [0, 0, {'VP': 0.008, 'Verb': 0.02, 'Preposition': 0.05}, 0, 0],
       [0, 0, 0, {'Det': 0.05}, 0],
       [0, 0, 0, 0, {'NP': 0.002, 'Nominal': 0.002, 'Noun': 0.01}]],
      dtype=object)

In [ ]:
# Iterar los elementos por encima de la diagonal principal para ir calculando 
# sus valores de Viterbi y actualizarlos en la matriz

for fila in range(0, cantidad_palabras):  
  for col in range(fila+1, cantidad_palabras):  

    #### Borrar (debug) ####
    print("Evaluando fila", fila, "- col", col, "->", Matriz[fila, col])  

    for colIzq in range(col):
      for filaAbajo in range(fila+1, cantidad_palabras):      
          CeldaIzq = Matriz[fila, colIzq]
          CeldaAbajo = Matriz[filaAbajo, col]      
          palabraIzq = Palabras[colIzq]
          palabraEval = Palabras[col]

          #### Borrar (debug) ####
          print("Palabra izq:", palabraIzq, "  Palabra Evaluada:", palabraEval)
          print("Celda Izq=", CeldaIzq, "  Celda abajo=", CeldaAbajo)

          Estado = dict()          
          # Si están vacías las celdas continuamos el ciclo y seguimos buscando
          if CeldaIzq ==0:
            print("Celda Izq nula", CeldaIzq)
            continue

          if CeldaAbajo==0:
            print("Celda Abajo nula", CeldaAbajo)
            continue

          # Checamos si hay una transición que genere un estado: Celda1 Celda2 -> E
          for estado1 in CeldaIzq:
            for estado2 in CeldaAbajo:
              try:
                # Si hay, convertir a lista la llave de Reglas[estado1][estado2] y tomamos el primer elemento
                # (porque es el estado de transición y con eso podemos tomar también su probabilidad)
                estado3 = list(Reglas[estado1][estado2].keys())[0]        
                probEstado1 = Reglas[palabraIzq][estado1]
                probEstado2 = Reglas[palabraEval][estado2]
                probEstado3 = Reglas[estado1][estado2][estado3]      
                probSintagma = probEstado1 * probEstado2 * probEstado3
                
                # Actualizar la matriz
                Estado.update({estado3: probSintagma})            
                Matriz[fila, col] = Estado

                #### Borrar (debug) ####  
                print("Palabra 1", palabraIzq, "->", estado1, "prob:", probEstado1)
                print("Palabra 2", palabraEval, "->", estado2, "prob:", probEstado2)                
                print(estado1, estado2,"->", estado3, "Prob:", probSintagma)

              except:
                # Cachamos la excepción por si no se encontró el elemento en el diccionario para continuar iterando
                estado3 = None
                probSintagma = 0

      #### Borrar (debug) #### 
      print("En[",fila,",",col,"] hay", Matriz[fila, col])
    print(" ")
    

Evaluando fila 0 - col 1 -> 0
Palabra izq: time   Palabra Evaluada: flies
Celda Izq= {'NP': 0.002, 'Nominal': 0.002, 'VP': 0.004, 'Verb': 0.01, 'Noun': 0.01}   Celda abajo= {'NP': 0.002, 'Nominal': 0.002, 'VP': 0.008, 'Verb': 0.02, 'Noun': 0.01}
Palabra 1 time -> NP prob: 0.002
Palabra 2 flies -> VP prob: 0.008
NP VP -> S Prob: 1.28e-05
Palabra 1 time -> Nominal prob: 0.002
Palabra 2 flies -> Nominal prob: 0.002
Nominal Nominal -> NP Prob: 8e-07
Palabra 1 time -> Nominal prob: 0.002
Palabra 2 flies -> Noun prob: 0.01
Nominal Noun -> Nominal Prob: 2.0000000000000003e-06
Palabra 1 time -> Verb prob: 0.01
Palabra 2 flies -> NP prob: 0.002
Verb NP -> VP Prob: 6e-06
Palabra izq: time   Palabra Evaluada: flies
Celda Izq= {'NP': 0.002, 'Nominal': 0.002, 'VP': 0.004, 'Verb': 0.01, 'Noun': 0.01}   Celda abajo= 0
Celda Abajo nula 0
Palabra izq: time   Palabra Evaluada: flies
Celda Izq= {'NP': 0.002, 'Nominal': 0.002, 'VP': 0.004, 'Verb': 0.01, 'Noun': 0.01}   Celda abajo= 0
Celda Abajo nula 0
Pa

In [ ]:
Matriz[0,4]

{'NP': 8e-07, 'Nominal': 2.0000000000000003e-06}